In [4]:
import re
import json
import spacy
import nltk
from nltk import CFG, ChartParser
from nltk.tokenize import word_tokenize

In [5]:
import re
import json
import pandas as pd 

In [6]:
VLMresponses = pd.read_csv("VLM_responses_Baseline_i.csv")
VLMresponses.head()

,UserPrompt,AssistantResponse,JSON,Accurate,Why it Failed?
0,I want to view traffic updates.,The user wants to view traffic updates. To com...,"{\n ""task"":""view traffic updates"",\n ""inte...",TRUE,NaN
1,I want to change the contact name to Jane Do.,To change the contact name to Jane Do using th...,"{\n ""task"":""change contact name to Jane Do"",...",TRUE,NaN
2,I want to add a red bag to my bag.,"Based on the image, the user would need to fol...",NaN,FALSE,NaN
3,I want to access my lessons offline.,"Based on the image, the user can complete the ...",NaN,FALSE,NaN
4,I want to give my answer verbally.,The image shows a mobile device displaying a u...,"{\n ""task"":""answer verbally and submit"",\n ...",TRUE,NaN


In [7]:
VLMresponses = VLMresponses[VLMresponses["Accurate"] != "FALSE"]

In [8]:
VLMresponses.head()

,UserPrompt,AssistantResponse,JSON,Accurate,Why it Failed?
0,I want to view traffic updates.,The user wants to view traffic updates. To com...,"{\n ""task"":""view traffic updates"",\n ""inte...",TRUE,NaN
1,I want to change the contact name to Jane Do.,To change the contact name to Jane Do using th...,"{\n ""task"":""change contact name to Jane Do"",...",TRUE,NaN
4,I want to give my answer verbally.,The image shows a mobile device displaying a u...,"{\n ""task"":""answer verbally and submit"",\n ...",TRUE,NaN
5,Can I maintain my current weight?,"Based on the image, to complete the task of 'C...","{\n ""task"":""maintain current weight"",\n ""i...",TRUE,NaN
6,Can you save my location?,"To save the user's location, the user should f...","{\n ""task"":""save location"",\n ""interaction...",TRUE,NaN


In [9]:
VLMresponses = VLMresponses.drop("Why it Failed?", axis='columns')
VLMresponses = VLMresponses.reset_index(drop=True)
VLMresponses.head()

,UserPrompt,AssistantResponse,JSON,Accurate
0,I want to view traffic updates.,The user wants to view traffic updates. To com...,"{\n ""task"":""view traffic updates"",\n ""inte...",TRUE
1,I want to change the contact name to Jane Do.,To change the contact name to Jane Do using th...,"{\n ""task"":""change contact name to Jane Do"",...",TRUE
2,I want to give my answer verbally.,The image shows a mobile device displaying a u...,"{\n ""task"":""answer verbally and submit"",\n ...",TRUE
3,Can I maintain my current weight?,"Based on the image, to complete the task of 'C...","{\n ""task"":""maintain current weight"",\n ""i...",TRUE
4,Can you save my location?,"To save the user's location, the user should f...","{\n ""task"":""save location"",\n ""interaction...",TRUE


In [10]:
# Sample Input
text = VLMresponses.iloc[22]["AssistantResponse"]
print(text)

jsonForm = VLMresponses.iloc[22]["JSON"]
print(jsonForm)


To complete the task of logging in with current credentials, the user should follow these steps based on the visible UI elements:

1. Tap on the "Enter your SplashTop Account" button to enter the email field.
2. Type in the email associated with the SplashTop account.
3. Tap on the "Password" field to enter the password.
4. Type in the password associated with the SplashTop account.
5. Tap on the "Log in" button to log in with the entered credentials.
{
   "task":"login with my current credentials",
   "interactions":[
      "Enter your SplashTop Account",
      "Log in",
      "email",
      "password"
   ]
}


In [11]:
def improved_semantic_parser(text):
    if not isinstance(text, str):
        return None

    # Try different ways to extract the task
    task_patterns = [
        r"wants to ([a-zA-Z\s]+@?)(?:.|\n|$)",
        r"To ([a-zA-Z\s]+@?)(?:.|\n|$)",
        r"task of ['\"]?([a-zA-Z\s]+@?)['\"]?(?:.|\n|$)",
        r"complete the task of ['\"]?([a-zA-Z\s]+@?)['\"]?(?:.|\n|$)",
        r"user (?:can|should) ([a-zA-Z\s]+@?)(?:.|\n|$)"
    ]

    task = None
    for pattern in task_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            task = match.group(1).strip().rstrip(".")
            break
        
    button_patterns = [
        r"Tap on the (.*?) button",
        r"Tap on the (.*?) icon",
        r"Tap on the (.*?) option",
        r"Press the (.*?) button",
        r"\b(\w+)\s+field",
        r"(\S+)\s+field",
    ]
    
    interactions = set()
    # Extract quoted UI elements as buttons
    for pattern in button_patterns:
        match = re.findall(pattern, text)
        
        if match:
            match = [m.strip().strip(r"'|\"") for m in match ] #if 1 <= len(m.strip().split()) <= 4
            for m in match:
                interactions.add(m.strip())
    
    # fields = set()
    # input_patterns = [
    #     r"\b(\w+)\s+field",
    #     r"(\S+)\s+field",
    # ]
    
    # for pattern in input_patterns:
    #     match = re.findall(pattern, text)
    #     if match:
    #         for m in match:
    #             fields.add(m.strip().strip(r"'|\""))
    

    return {
        "task": task,
        "interactions": list(interactions)
    }

In [12]:
print("\n Semantic Parser Output ")
print(json.dumps(improved_semantic_parser(text), indent=3))


 Semantic Parser Output 
{
   "task": "complete the task of logging in with current credentials",
   "buttons": [
      "Log in",
      "Enter your SplashTop Account"
   ],
   "field": [
      "email",
      "Enter your SplashTop Account",
      "Log in",
      "Password"
   ]
}


In [13]:
VLMresponses["SemanticParser"] = VLMresponses["AssistantResponse"].apply(improved_semantic_parser)
VLMresponses.head()
VLMresponses.to_csv("VLM_responses_Baseline_SemanticParser.csv", index=False)

In [26]:
VLMTrainingResponses = pd.read_csv("VLMTrainingResponses.csv")
VLMTrainingResponses.head()

,Index,UserPrompt,AssistantResponse
0,0,click on the image beside the option traffic,The user wants to click on the image beside th...
1,1,click on green color button,"To click on the green color button, the user s..."
2,2,share,"To share the product from the screenshot, the ..."
3,3,click on see plans,"To complete the task of clicking on ""See Plans..."
4,4,click on the mike,"To click on the mike icon, the user should fol..."


In [27]:
VLMTrainingResponses["SemanticParser"] = VLMTrainingResponses["AssistantResponse"].apply(improved_semantic_parser)
VLMTrainingResponses.head()

,Index,UserPrompt,AssistantResponse,SemanticParser
0,0,click on the image beside the option traffic,The user wants to click on the image beside th...,{'task': 'click on the image beside the option...
1,1,click on green color button,"To click on the green color button, the user s...","{'task': 'click on the green color button', 'i..."
2,2,share,"To share the product from the screenshot, the ...",{'task': 'share the product from the screensho...
3,3,click on see plans,"To complete the task of clicking on ""See Plans...","{'task': 'complete the task of clicking on', '..."
4,4,click on the mike,"To click on the mike icon, the user should fol...","{'task': 'click on the mike icon', 'interactio..."


In [ ]:
VLMTrainingResponses.to_csv("VLM__training_responses.csv", index=False)